In [100]:
from haystack.nodes import FARMReader, TransformersReader
from haystack.utils import print_answers

In [101]:
from haystack.document_stores import ElasticsearchDocumentStore

    # Connect to Elasticsearch
document_store = ElasticsearchDocumentStore(host="localhost", username="", password="", index="auto_insurance")

In [102]:
from haystack.nodes import DensePassageRetriever

In [103]:
retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    max_seq_len_query=64,
    max_seq_len_passage=256,
    batch_size=16,
    use_gpu=True,
    embed_title=True,
    use_fast_tokenizers=True,
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  =============
INFO - haystack.modeling.model.language_model -  Could not find facebook/dpr-question_encoder-single-nq-base locally.
INFO - haystack.modeling.model.language_model -  Looking on Transformers Model Hub (in local cache and online)...
INFO - haystack.modeling.model.language_model -  Loaded facebook/dpr-question_encoder-single-nq-base
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'DPRQuestionEncoderTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.
INFO - haystack.modeling.model.language_model -  LOADING MODEL
INFO - haystack.modeling.model.language_model -  ======

In [104]:
document_store.update_embeddings(retriever)

INFO - haystack.document_stores.elasticsearch -  Updating embeddings for all 408 docs ...


Updating embeddings:   0%|          | 0/408 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/416 [00:00<?, ? Docs/s]

In [105]:
reader = TransformersReader(model_name_or_path="distilbert-base-uncased-distilled-squad", tokenizer="distilbert-base-uncased", use_gpu=-1)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


In [106]:
from haystack.pipelines import ExtractiveQAPipeline

pipe = ExtractiveQAPipeline(reader, retriever)

In [133]:
prediction = pipe.run(
    query="what is the premium for my insured auto?", params={"Retriever": {"top_k": 10}, "Reader": {"top_k": 5}}
)

In [134]:
print_answers(prediction, details="minimum")


Query: what is the premium for my insured auto?
Answers:
[   {   'answer': '$10',
        'context': 'Allstate will repay you up to $10 for the cost of '
                   'transportation from the place of theft of your insure'},
    {   'answer': '$50 or less',
        'context': 'your insured auto and another auto insured by us, even if '
                   'the loss is $50 or less.'},
    {   'answer': 'comprehensive insurance',
        'context': 'on, falling objects, fire, lightning, or flood and you '
                   'have purchased comprehensive insurance.'},
    {   'answer': 'All amounts',
        'context': 'All amounts paid by the owner or operator of the uninsured '
                   'auto or anyone else re'},
    {   'answer': 'bodily injury liability',
        'context': 'This includes all sums paid under the bodily injury '
                   'liability coverage of this or any other auto policy.'}]
